# 顔文字の抽出

In [1]:
!pip install nagisa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 48.8 MB/s eta 0:00:00


In [16]:
KAOMOJI_MIN = 4
KAOMOJI_MAX = 15

In [17]:
import nagisa
import unicodedata


def extract_kaomoji(text):
  words = nagisa.extract(text, extract_postags=['補助記号']).words

  if len(words) == 0:
    return None

  max_len = len(max(words, key=len))
  if max_len < KAOMOJI_MIN:
    return None

  while words[-2:] == ['[', ']']:
    words = words[:-2]

  result = ''.join(words)
  if len(result) > KAOMOJI_MAX:
    return None

  return result

In [18]:
in_files = ['simeji', 'kmoji', 'kaomoji_copy']

In [19]:
for name in in_files:
  with open(name + '_extracted.txt', mode='w', encoding='utf-8') as out_file:
    with open(name + '.txt', mode='r') as in_file:
      for line in in_file:
        result = extract_kaomoji(line)
        if result != None:
          out_file.write(result + '\n')

## ファイルをまとめる

In [20]:
kaomoji_data = []

for name in in_files:
  with open(name + '_extracted.txt', mode='r') as in_file:
    for line in in_file:
      kaomoji_data.append(line)

In [21]:
with open('kaomoji_MAX=' + str(KAOMOJI_MAX) + '.txt', \
           mode='w', encoding='utf-8') as out_file:
  for data in list(set(kaomoji_data)):
    out_file.write(data)